In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Finally now, the Analytics ..

In [ ]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


In [ ]:

#  Connected components ..


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "pagerank_" + str(l_cntr).zfill(4)


analytics.pagerank(
   my_graph,
   result_property_name = l_output_propname,
   max_iterations       = 10000
   )

print("--")


In [ ]:

#  Unfiltered node data
#

l_query = """

   MATCH (n)
   RETURN n.{0}, ID(n), n
   ORDER BY n.{0} DESC
    
    """.format(l_output_propname)

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data,
#
#     +----+-------------------+---------+-----------------+-------------+-----------+------------------+------------------------+-------------------+----------+
#     |    |   n.pagerank_0021 |   ID(n) |   n.internal_id | n.labels    | n.LABEL   | n.airport_code   | n.airport_name         |   n.pagerank_0021 | n.type   |
#     |----+-------------------+---------+-----------------+-------------+-----------+------------------+------------------------+-------------------+----------|
#     |  0 |          1.62373  |       3 |               3 | ['Airport'] | Airport   | DEN              | Denver                 |          1.62373  | node     |
#     |  1 |          1.48703  |       1 |               1 | ['Airport'] | Airport   | ORD              | Chicago O-Hare         |          1.48703  | node     |
#     |  2 |          0.894602 |       4 |               4 | ['Airport'] | Airport   | MSP              | Minneapolis, St.Paul   |          0.894602 | node     |
#     |  3 |          0.894602 |       5 |               5 | ['Airport'] | Airport   | MSN              | Madison                |          0.894602 | node     |
#     |  4 |          0.894602 |       6 |               6 | ['Airport'] | Airport   | GRB              | Green Bay              |          0.894602 | node     |
#     |  5 |          0.894602 |       0 |               0 | ['Airport'] | Airport   | MKE              | Milwaukee              |          0.894602 | node     |
#     |  6 |          0.884786 |       7 |               7 | ['Airport'] | Airport   | EGE              | Eagle, Vail            |          0.884786 | node     |
#     |  7 |          0.884786 |       8 |               8 | ['Airport'] | Airport   | COS              | Colorado Springs       |          0.884786 | node     |
#     |  8 |          0.884786 |       2 |               2 | ['Airport'] | Airport   | SJC              | San Jose               |          0.884786 | node     |
#     |  9 |          0.884786 |       9 |               9 | ['Airport'] | Airport   | PUB              | Pueblo                 |          0.884786 | node     |
#     | 10 |          0.884786 |      10 |              10 | ['Airport'] | Airport   | ASE              | Aspen                  |          0.884786 | node     |
#     | 11 |          0.884786 |      11 |              11 | ['Airport'] | Airport   | FNL              | Fort Collins, Loveland |          0.884786 | node     |
#     +----+-------------------+---------+-----------------+-------------+-----------+------------------+------------------------+-------------------+----------+


In [ ]:

#  Nodes and a count of outgoing edges
#
l_query = """

   MATCH (n) - [r] -> (m)
   RETURN n.airport_code, n.airport_name, COUNT(r) AS my_outgoing_edges
   ORDER BY my_outgoing_edges DESC
    
    """.format(l_output_propname)

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample data,
#
#     +----+------------------+------------------------+---------------------+
#     |    | n.airport_code   | n.airport_name         |   my_outgoing_edges |
#     |----+------------------+------------------------+---------------------|
#     |  0 | DEN              | Denver                 |                   7 |
#     |  1 | ORD              | Chicago O-Hare         |                   5 |
#     |  2 | SJC              | San Jose               |                   2 |
#     |  3 | EGE              | Eagle, Vail            |                   1 |
#     |  4 | ASE              | Aspen                  |                   1 |
#     |  5 | COS              | Colorado Springs       |                   1 |
#     |  6 | MKE              | Milwaukee              |                   1 |
#     |  7 | MSP              | Minneapolis, St.Paul   |                   1 |
#     |  8 | FNL              | Fort Collins, Loveland |                   1 |
#     |  9 | GRB              | Green Bay              |                   1 |
#     | 10 | MSN              | Madison                |                   1 |
#     | 11 | PUB              | Pueblo                 |                   1 |
#     +----+------------------+------------------------+---------------------+
